In [1]:
using ParProx, Printf, Statistics # load the packages
using CSV, DataFrames, CodecZlib, Mmap # packages for data reading. GZip is used to read the gzipped text file.

# package for one-hot encoding
using NPZ
using Pickle
using JSON

[ Info: Precompiling ParProx [440725ae-e20d-4dd2-817a-b87798dc3ed7]


In [2]:
using CUDA, Adapt

In [3]:
sort_order = npzread("preprocessed/sort_order.npz")["data"];
survival = npzread("preprocessed/survival.npz");
survival_event = survival["pfi"];
survival_time = survival["pfi_time"];

In [4]:
# group_to_variables = [convert(Vector{Int64}, x) for x in Pickle.load("preprocessed/group_to_variables_index.pkl")];
# array_predictors = npzread("preprocessed/array_predictors.npz")["data"];
# json_file_path = "preprocessed/index_to_group.json"
# index_to_group = JSON.parsefile(json_file_path);

# json_file_path = "preprocessed/index_to_variable.json"
# index_to_variable = JSON.parsefile(json_file_path);

# json_file_path = "preprocessed/variable_to_index.json"
# variable_to_index = JSON.parsefile(json_file_path);

In [5]:
group_to_variables = [convert(Vector{Int64}, x) for x in Pickle.load("only_mrna/group_to_variables_index.pkl")];
array_predictors = npzread("only_mrna/array_predictors.npz")["data"];
json_file_path = "only_mrna/index_to_group.json"
index_to_group = JSON.parsefile(json_file_path);

json_file_path = "only_mrna/index_to_variable.json"
index_to_variable = JSON.parsefile(json_file_path);

json_file_path = "only_mrna/variable_to_index.json"
variable_to_index = JSON.parsefile(json_file_path);

In [6]:
X = array_predictors[:, 1:end-5]
X_unpen = array_predictors[:, end-4:end]
size(X), size(X_unpen)

((726, 20531), (726, 5))

In [7]:
lambdas = 10 .^ (range(-8, stop=-10, length=21))

21-element Vector{Float64}:
 1.0e-8
 7.943282347242822e-9
 6.309573444801943e-9
 5.011872336272715e-9
 3.981071705534969e-9
 3.1622776601683795e-9
 2.511886431509582e-9
 1.9952623149688828e-9
 1.584893192461111e-9
 1.2589254117941663e-9
 1.0e-9
 7.943282347242822e-10
 6.309573444801942e-10
 5.011872336272714e-10
 3.9810717055349694e-10
 3.1622776601683795e-10
 2.511886431509582e-10
 1.9952623149688828e-10
 1.584893192461111e-10
 1.2589254117941662e-10
 1.0e-10

In [8]:
# CUDA.allowscalar(true)

# T = Float64
# A = CuArray
# U = ParProx.COXUpdate(; maxiter=20000, step=1000, tol=1e-6, verbose=true)

# @time scores = ParProx.cross_validate(
#     U,
#     adapt(A{T}, X),
#     adapt(A{T}, X_unpen),
#     adapt(A{T}, survival_event),
#     adapt(A{T}, survival_time),
#     group_to_variables,
#     lambdas, 5;
#     T=Float64
# )

In [9]:
# lambda_idx = argmax(mean(scores; dims=2)[:])
# lambda = lambdas[lambda_idx]

In [10]:
lambda = lambdas[10]

1.2589254117941663e-9

In [11]:
CUDA.allowscalar(true)

T = Float64
A = CuArray
U = ParProx.COXUpdate(; maxiter=20000, step=10, tol=1e-12, verbose=true)
V = ParProx.COXVariables{T}(
    adapt(A{T}, X),
    adapt(A{T}, X_unpen),
    adapt(A{T}, survival_event),
    adapt(A{T}, survival_time),
    4.3e-8,
    group_to_variables,
    eval_obj=true
)

@time ParProx.fit!(U, V)

v.σ = 3.6275709043856147e-8


LoadError: type COXVariables has no field obj

In [ ]:
_, grpmat, _ = ParProx.mapper_mat_idx(group_to_variables, length(index_to_variable));

In [ ]:
β_orig = vcat(grpmat * collect(V.β[1:end-5]), collect(V.β)[end-4:end]);

In [ ]:
variable_names_replicated = String[]
for i in 1:length(group_to_variables)
    for v in group_to_variables[i]
        variable_name = index_to_variable[string(v)]
        # print(variable_name)
        push!(variable_names_replicated, "$variable_name")
    end
end

In [ ]:
variable_names_replicated = [variable_names_replicated; "age_at_initial_pathologic_diagnosis"; "gender"; "BLACK OR AFRICAN AMERICAN"; "ASIAN"; "AMERICAN INDIAN OR ALASKA NATIVE"];

In [ ]:
df_variable_name = DataFrame(index = collect(keys(index_to_variable)), value = collect(values(index_to_variable)))
df_variable_name.index = parse.(Int, df_variable_name.index);
sort!(df_variable_name, :index);
variable_names = [df_variable_name.value; "age_at_initial_pathologic_diagnosis"; "gender"; "BLACK OR AFRICAN AMERICAN"; "ASIAN"; "AMERICAN INDIAN OR ALASKA NATIVE"];

In [ ]:
for (v, β) in zip(variable_names_replicated[V.β .!= 0], V.β[V.β .!= 0])
    println("$v\t$β")
end

In [ ]:
for (v, β) in zip(variable_names[β_orig .!= 0], β_orig[β_orig .!= 0])
    println("$v\t$β")
end